In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import requests

sns.set()

calculations of averages on scb/fhm data will not exactly correspond to the official numbers. Probable reason
is that for SCB data, my granularity is 1 year, for FHM data 10 year bins. Consider the diff between someone
born 1 jan, dying 1 jan next year. That will be counted in the 1 year old bin. Now, consider someone born dec last,also dying jan 1:st next year. That person could be counted as 0 or 1 years old, depending on how the scb stats are organized, which I don't know. In any case, I bet for scb offical stat's, they count with year fractions, while I in the best case have integer data, with 1 year granularity at best. Thus, my averages should therefore be higher than those of scb. 

In [ ]:
### deaths ####

dead_url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'
dead_json = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r_dead = requests.post(dead_url,json=dead_json)
r_dead.status_code

In [ ]:
json_dead = r_dead.json()
json_dead['data'][0]

In [ ]:
### population ###
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101A/BefolkningR1860'
data = {
  "query": [
    {
      "code": "Alder",
      "selection": {
        "filter": "agg:Ålder5år",
        "values": [
          "-4",
          "5-9",
          "10-14",
          "15-19",
          "20-24",
          "25-29",
          "30-34",
          "35-39",
          "40-44",
          "45-49",
          "50-54",
          "55-59",
          "60-64",
          "65-69",
          "70-74",
          "75-79",
          "80-84",
          "85-89",
          "90-94",
          "95-99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    },
    {
      "code": "Tid",
      "selection": {
        "filter": "item",
        "values": [
          "1968",
          "1969",
          "1970",
          "1971",
          "1972",
          "1973",
          "1974",
          "1975",
          "1976",
          "1977",
          "1978",
          "1979",
          "1980",
            "1981",
            "1982",
            "1983",
            "1984",
            "1985",
            "1986",
            "1987",
            "1988",
            "1989",
            "1990",
            "1991",
            "1992",
            "1993",
            "1994",
            "1995",
            "1996",
            "1997",
            "1998",
            "1999",
            "2000",
            "2001",
            "2002",
            "2003",
            "2004",
            "2005",
            "2006",
            "2007",
            "2008",
            "2009",
            "2010",
            "2011",
            "2012",
            "2013",
          "2014",  
          "2015",
          "2016",
          "2017",
          "2018",
          "2019"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

r = requests.post(url,json=data)
print (r.status_code)
json_pop = r.json()

json_pop['data'][0]

In [ ]:
def parse_json(json):
    
    nr_keys = len(json['data'][0]['key'])
    nr_vals = len(json['data'][0]['values'])
    
    
    items = len(json['data'])
    data_list = []

    for i in range(items):
        val_list = []
        
        for j,k in enumerate(range(nr_keys)):
            val_list.append(json['data'][i]['key'][j])
            
        for j,v in enumerate(range(nr_vals)):
            val_list.append(json['data'][i]['values'][j])
            
        data_list.append(val_list)

    return data_list
    
dead_df = pd.DataFrame(parse_json(json_dead))
dead_df.drop(0,axis=1,inplace=True)
dead_df    

In [ ]:
dead_df.columns = ['age','gender','year','dead']
dead_df['age'] = pd.Categorical(dead_df['age'],['-4','5-9','10-14','15-19','20-24',
                                                 '25-29','30-34','35-39','40-44',
                                                 '45-49','50-54','55-59','60-64','65-69',
                                                 '70-74','75-79','80-84','85-89',
                                                 '90-94','95-99','100+'])

dead_df['dead'] = dead_df['dead'].astype(int)
dead_df

In [ ]:
dead_df_men_women = dead_df.groupby(['year','age']).sum()

dead_df_men_women = dead_df_men_women.unstack()
dead_df_men_women = dead_df_men_women.droplevel(0,axis=1)
dead_df_men_women.info()

In [ ]:
dead_df_men_women

In [ ]:
pop_df = pd.DataFrame(parse_json(json_pop))
pop_df.columns = ['age','gender','year','pop']
pop_df['pop'] = pop_df['pop'].astype(int)
pop_df['age'] = pd.Categorical(pop_df['age'],['-4','5-9','10-14','15-19','20-24',
                                                 '25-29','30-34','35-39','40-44',
                                                 '45-49','50-54','55-59','60-64','65-69',
                                                 '70-74','75-79','80-84','85-89',
                                                 '90-94','95-99','100+'])

pop_df

In [ ]:
pop_df = pop_df.groupby(['year','age']).sum()
pop_df

In [ ]:
pop_df_men_women = pop_df.unstack().droplevel(0,axis=1)
pop_df_men_women

In [ ]:

### NOT REAL MORTALITY SINCE WE ARE ADDING MORTALITIES FOR DIFFERENT AGE GROUPS ! ###

age_grp_mortality = dead_df_men_women / pop_df_men_women
age_grp_mortality

In [ ]:
avg_mortality_per_age_grp = age_grp_mortality.mean()
avg_mortality_per_age_grp

In [ ]:
avg_mortality_per_age_grp.plot(figsize=(18,12),style='o--')

In [ ]:
std_pop = pop_df_men_women.loc['2019']

In [ ]:
strata_abs_deaths = age_grp_mortality * std_pop
strata_abs_deaths

In [ ]:
strata_abs_deaths

In [ ]:
strata_abs_deaths.plot(kind='bar',stacked=True,figsize=(18,12))

In [ ]:
tot_strat_deaths = strata_abs_deaths.sum(axis=1)
tot_strat_deaths

In [ ]:
age_adj_mortality = tot_strat_deaths / std_pop.sum()
age_adj_mortality

In [ ]:
age_adj_mortality.plot(kind='bar',figsize=(18,12))
_= plt.yticks(np.arange(0,0.019,0.001))

In [ ]:
dead_df_men_women.head()

In [ ]:
dead_2019 = pd.DataFrame(dead_df_men_women.loc['2019'])
dead_2019['age_idx'] = range(1,len(dead_2019) + 1)
dead_2019


In [ ]:
dead_2019.describe()

In [ ]:
ticks_2019 = pd.Series(np.repeat(dead_2019['age_idx'],dead_2019['2019']))
ticks_2019 = ticks_2019.reset_index()
ticks_2019

In [ ]:
ticks_2019.describe()

In [ ]:
dead_df_men_women.columns

In [ ]:
### https://la.utexas.edu/users/denbow/labs/Ageofdeath.htm ###


avg_age_of_death = dead_df_men_women.copy()
# first, calculate median age per age group by subtracting the larger number, dividing by 2,
# then adding that to the lower nr.

grp_medians = [(4-0)/2+0,(9-5)/2+5,(14-10)/2+10,(19-15)/2+15,(24-20)/2+20,(29-25)/2+25,
              (34-30)/2+30,(39-35)/2+35,(44-40)/2+40,(49-45)/2+45,(54-50)/2+50,(59-55)/2+55,
              (64-60)/2+60,(69-65)/2+65,(74-70)/2+70,(79-75)/2+75,(84-80)/2+80,(89-85)/2+85,
              (94-90)/2+90,(99-95)/2+95,100]
grp_medians

In [ ]:
avg_age_of_death.columns=grp_medians
avg_age_of_death

In [ ]:
yearly_sums = avg_age_of_death.sum(axis=1)
yearly_sums

In [ ]:
years_lost = avg_age_of_death * grp_medians
tot_years_lost = years_lost.sum(axis=1)
tot_years_lost

In [ ]:
avg_age = tot_years_lost / yearly_sums
avg_age

In [ ]:
avg_age.plot()

In [ ]:
avg_age / avg_age.shift()

In [ ]:
### dead in yearly groups 
url = 'http://api.scb.se/OV0104/v1/doris/sv/ssd/START/BE/BE0101/BE0101I/DodaFodelsearK'
data = {
  "query": [
    {
      "code": "Region",
      "selection": {
        "filter": "vs:RegionRiket99",
        "values": [
          "00"
        ]
      }
    },
    {
      "code": "Alder",
      "selection": {
        "filter": "vs:Ålder1årA",
        "values": [
          "0",
          "1",
          "2",
          "3",
          "4",
          "5",
          "6",
          "7",
          "8",
          "9",
          "10",
          "11",
          "12",
          "13",
          "14",
          "15",
          "16",
          "17",
          "18",
          "19",
          "20",
          "21",
          "22",
          "23",
          "24",
          "25",
          "26",
          "27",
          "28",
          "29",
          "30",
          "31",
          "32",
          "33",
          "34",
          "35",
          "36",
          "37",
          "38",
          "39",
          "40",
          "41",
          "42",
          "43",
          "44",
          "45",
          "46",
          "47",
          "48",
          "49",
          "50",
          "51",
          "52",
          "53",
          "54",
          "55",
          "56",
          "57",
          "58",
          "59",
          "60",
          "61",
          "62",
          "63",
          "64",
          "65",
          "66",
          "67",
          "68",
          "69",
          "70",
          "71",
          "72",
          "73",
          "74",
          "75",
          "76",
          "77",
          "78",
          "79",
          "80",
          "81",
          "82",
          "83",
          "84",
          "85",
          "86",
          "87",
          "88",
          "89",
          "90",
          "91",
          "92",
          "93",
          "94",
          "95",
          "96",
          "97",
          "98",
          "99",
          "100+"
        ]
      }
    },
    {
      "code": "Kon",
      "selection": {
        "filter": "item",
        "values": [
          "1",
          "2"
        ]
      }
    }
  ],
  "response": {
    "format": "json"
  }
}

In [ ]:
r = requests.post(url,json=data)
r.status_code

In [ ]:
json = r.json()

In [ ]:
json['data'][0]

In [ ]:
dead_yearly_grp = pd.DataFrame(parse_json(json))
dead_yearly_grp.drop(0,axis=1,inplace=True)
dead_yearly_grp.columns = ['age','gender','year','dead']
dead_yearly_grp['dead'] = dead_yearly_grp['dead'].astype(int)

dead_by_gender = dead_yearly_grp.copy()

dead_yearly_grp = (dead_yearly_grp.groupby(['year','age']).sum()).reset_index()
dead_yearly_grp

In [ ]:
# to test effect 
#dead_yearly_grp['age'] = dead_yearly_grp['age'].replace('100+','120')

In [ ]:
dead_yearly_grp['age'] = dead_yearly_grp['age'].apply(lambda x : x.replace('+','')).astype(int)
# test effect 
#dead_yearly_grp['age'] += 0.5

# test_effect
#dead_yearly_grp = dead_yearly_grp.loc[dead_yearly_grp['age'] > 0]

dead_yearly_grp

In [ ]:
dead_yearly_grp = dead_yearly_grp.sort_values(['year','age'])

yearly_deaths = dead_yearly_grp.copy()
yearly_deaths

In [ ]:
dead_yearly_grp = dead_yearly_grp.groupby(['year','age']).mean()
dead_yearly_grp

In [ ]:
dead_yearly_grp['years_lived'] = dead_yearly_grp.index.get_level_values(1) * dead_yearly_grp['dead']
dead_yearly_grp

In [ ]:
totals = dead_yearly_grp.groupby('year').sum()
totals

In [ ]:
avg_age_2 = totals['years_lived'] / totals['dead']
avg_age_2

In [ ]:
ax = avg_age.plot(alpha=0.5,figsize=(18,12),label='avg_age')

# by adding 1 to each age group, as above is test effect, the diff is significant

avg_age_2.plot(ax=ax,alpha=0.5,label='avg_age_2')
plt.legend(loc='upper left')

To get median from a frequency table, start by sorting the keys (age groups) into ascending scale. Then do a 
cumsum on the frequency (dead). Take the median (midpoint) of that cumsum, that is, the half value of the cumsum. Then find out which value in the cumsum column corresponds to that half cumsum (median)

https://support.sisense.com/hc/en-us/community/posts/360033174073-Calculating-Weighted-Median-Median-of-Data-in-Frequency-Table-Mean-Mode

In [ ]:
g = dead_yearly_grp.groupby('year',group_keys=False)
g

In [ ]:
sorted_dead = g.apply(lambda x : x.sort_index(level=[0,1],ascending=[True,True]))
sorted_dead

In [ ]:
cums = sorted_dead.groupby('year').cumsum()
cums.columns = ['cum_dead','cum_years_lived']
cums

In [ ]:
all_dead = sorted_dead.join(cums)
all_dead

In [ ]:
all_dead.drop(['years_lived','cum_years_lived'],axis=1,inplace=True)
all_dead

In [ ]:
# median is middle value of cum_dead #
medians = all_dead.groupby('year')['cum_dead'].apply(lambda x : x.iloc[-1] / 2 + 1) # ODD NR OF DATA POINTS
medians

In [ ]:
foo = all_dead['cum_dead'].unstack()
foo = foo.T
foo

In [ ]:
foo - medians

In [ ]:
baz = foo >= medians
median_age = len(foo) - baz.sum()
median_age

In [ ]:
mask = (foo - medians) >= 0
bar = foo[mask]

bar

In [ ]:
for col in bar.columns:
    print (col,bar[col].argmin())

In [ ]:
all_dead
all_dead_2019 = all_dead.loc['2019']
all_dead_2019

In [ ]:
median = all_dead_2019.iloc[-1]['cum_dead'] / 2+1
median

In [ ]:
all_dead_2019.loc[all_dead_2019['cum_dead'] >= median]

In [ ]:
# simple ex 


df = pd.DataFrame({'score' : range(1,6),
                  'freq' :range(1,6)})

df['cum_freq'] = df['freq'].cumsum()
median_point = int(df['cum_freq'].iloc[-1] / 2 + 1)

print (median_point)

df
# so median is the 8th score, which falls in cum_freq value row 10, i.e the 8:th value is a score of 4

In [ ]:
flat_scores = pd.Series(np.repeat(df['score'],df['freq']))
flat_scores = flat_scores.reset_index(drop=True)
flat_scores.median()

In [ ]:
# flat on real data

In [ ]:
all_dead


In [ ]:
all_dead['age_col'] = all_dead.index.get_level_values(1)
all_flat = all_dead.groupby('year',group_keys=False).apply(lambda row : np.repeat(row['age_col'],row['dead']))
all_flat

In [ ]:
flattened_medians = all_flat.groupby('year').median()
flattened_means = all_flat.groupby('year').mean()
flats = pd.concat([flattened_medians,flattened_means],axis=1)
flats.columns=['median','mean']
flats

In [ ]:
flats.plot(figsize=(18,12),style='o--')

In [ ]:
all_dead.droplevel(1).groupby('year')['dead'].sum()

vals = [1,2,3,4,5]

np.random.choice(vals,5,replace=False)

In [ ]:
# general example of descriptive stats taken from a frequency table

np.random.seed(666)

vals = [0,1,2,3,4,5]

nr_categories = 5

df_freq = pd.DataFrame({'math' : np.random.choice(vals,nr_categories,replace=False),
                  'physics' : np.random.choice(vals,nr_categories,replace=False),
                  'chemistry' : np.random.choice(vals,nr_categories,replace=False),
                  'biology' : np.random.choice(vals,nr_categories,replace=False)},
                       index=range(1,nr_categories + 1))

df_freq.index.name='points'
df_freq.columns.name='subject'

df_freq.sort_index(inplace=True)

# taking mean & median does not give what we want - it gives values of freq, not of points

print ()
print ('mean',df_freq.mean())
print ()
print ('median',df_freq.median())
print ()
df_freq

In [ ]:
df_freq.sum()

In [ ]:
df_freq_subgrps = pd.DataFrame(df_freq.unstack()) 
df_freq_subgrps.columns=['freq']
df_freq_subgrps['points'] = df_freq_subgrps.index.get_level_values(1)
df_freq_subgrps = df_freq_subgrps[['points','freq']]
df_freq_subgrps = df_freq_subgrps.droplevel(1)

df_freq_subgrps

In [ ]:
serialized_points = df_freq_subgrps.groupby('subject',group_keys=False).apply(
        lambda row: np.repeat(row['points'],row['freq'])) # flat is a Series

serialized_points

In [ ]:
serialized_points['math'].agg(['median','mean'])

In [ ]:
stats = serialized_points.groupby('subject').agg(['median','mean','count','min','max'])
stats

In [ ]:

dead_by_gender['age'] = dead_by_gender['age'].apply(lambda x : x.strip('+')).astype(int)
dead_by_gender

In [ ]:
gender_groups = dead_by_gender.groupby(['year','age','gender']).apply(lambda row : np.repeat(row.age,row.dead))
gender_groups

In [ ]:
gender_stats = gender_groups.groupby(['year','gender']).agg(['median','mean','count'])
gender_stats

In [ ]:
unstacked_gender_stats = gender_stats.reset_index()
unstacked_gender_stats

In [ ]:
fig,ax = plt.subplots(figsize=(18,12))
sns.barplot(x='year',y='mean',hue='gender',data=unstacked_gender_stats,ax=ax)
_=plt.xticks(rotation=90)

In [ ]:
common_stats = gender_groups.groupby('year').agg(['median','mean','count'])
common_stats

In [ ]:
ax = common_stats.plot(figsize=(18,12))

#flats.plot(ax=ax)

In [ ]:
yearly_deaths = yearly_deaths.set_index('year')
yearly_deaths

In [ ]:
#yearly_deaths.drop('years_lived',axis=1,inplace=True)

In [ ]:
#yearly_deaths['age_num'] = yearly_deaths.index.get_level_values(1)
#yearly_deaths

In [ ]:
freq_1973 = yearly_deaths.loc['1973']
serialized_1973 = freq_1973.groupby('age',group_keys=False).apply(lambda row : np.repeat(row.age,row.dead))
serialized_1973.plot(kind='hist',bins=range(0,101))


In [ ]:
# 1973 has max freq shared by 77 & 78 years, dual mode
serialized_1973.mode()

In [ ]:
serialized_yearly_deaths = yearly_deaths.groupby(
    'year',group_keys=False).apply(lambda row : np.repeat(row.age,row.dead))

print (serialized_yearly_deaths)

death_stats = serialized_yearly_deaths.groupby('year').agg(['mean','median','count'])
death_stats

In [ ]:
# since 1973 has dual mode, we take the last (highest) value

death_mode = serialized_yearly_deaths.groupby('year').apply(lambda row : row.mode().iloc[-1])

death_mode



In [ ]:
death_stats['mode'] = death_mode
death_stats

In [ ]:
excel = pd.ExcelFile('fhm_excel.xlsx')
excel.sheet_names

In [ ]:
age_df = excel.parse('Totalt antal per åldersgrupp')
age_df = age_df[:-1]
age_grp_median = np.arange(5,100,10)
age_df['age_num'] = age_grp_median
age_df


In [ ]:
fhm_dead_serialized = age_df.groupby('Åldersgrupp').apply(
    lambda row : np.repeat(row['age_num'],row['Totalt_antal_avlidna']))

fhm_dead_serialized = fhm_dead_serialized.droplevel(1)
fhm_dead_serialized

In [ ]:
fhm_dead_mode = fhm_dead_serialized.mode()
fhm_dead_mode

In [ ]:
fhm_dead_stats = fhm_dead_serialized.agg(['mean','median','count'])
fhm_dead_stats

In [ ]:
colors = ['red','green','blue']

ax = death_stats.plot(figsize=(18,12),style=['ro--','go--','bo--'],y=['mean','median','mode'],
                 title='SWEDEN 1968 - 2019 age of death [all-cause] stats\nDataSource : SCB.se')


xtick_pos = plt.xticks()
xtick_pos[0]

props = dict(boxstyle='round', facecolor='wheat', alpha=0.5)



plt.text(0,88.5,'2020 COVID STATS:',color='crimson',size=18,bbox=props)
plt.text(0,87,'* mean age of death: {:.0f}'.format(fhm_dead_stats['mean']),color='red',bbox = props)
plt.text(0,86.5,'* median age of death: {:.0f}'.format(fhm_dead_stats['median']),color='green',bbox=props)
plt.text(0,86,'* mode age of death: {:.0f}'.format(fhm_dead_mode.iloc[0]),color='blue',bbox=props)

plt.ylabel('age')

plt.yticks(range(70,95,1))
_= plt.xticks(range(len(death_stats)),death_stats.index,rotation=90)

plt.savefig('swe_median_mean_mode_age_of_death_cmp_covid.jpg',format='jpg')

In [ ]:
all_dead

In [ ]:
dead_2001 = all_dead.loc['2001']
dead_2000 = all_dead.loc['2000']
dead_2001

In [ ]:
ax = dead_2001.plot(kind='bar',figsize=(18,12),y='dead',color='red',alpha=0.5)
dead_2000.plot(kind='bar',y='dead',ax=ax,color='g',alpha=0.5)
plt.legend(['dead 2001','dead 2000'])

In [ ]:
ax = dead_2001['cum_dead'].plot(figsize=(18,12),title='SWE 2001 & 2000 cumulative deaths')
dead_2000['cum_dead'].plot(ax=ax)
plt.legend(['2001 cumulative dead','2000 cumulative dead'])
plt.ylabel('nr deaths')

In [ ]:
unstacked_gender_stats


In [ ]:
male = unstacked_gender_stats.loc[unstacked_gender_stats['gender'] == '1']
female = unstacked_gender_stats.loc[unstacked_gender_stats['gender'] == '2']

ax = male.plot(x='year',figsize=(18,12),style='o--',y=['median','mean'])
female.plot(ax=ax,x='year',style='o--',y=['median','mean'])

plt.yticks(range(68,87))
ax.legend(['male median','male mean','female median','female mean'])